Runs an extended Kalman filter on Prof Flaxman's SEIIR predictions and
measurement data for New York State from mid-April to 7 July 2020.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [1]:
#%load_ext autoreload
#%autoreload

In [2]:
#%reset

In [3]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets
import seiir_compartmental

In [4]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]]])

    beta_k = seiir['beta_pred'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2]])

    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0]])

    # 5x5
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R], axis=1)

    # 5x5
    # P_k1_prior = f_jacob * P * f_jacob^T + Q
    P_k1_prior = np.matmul(np.matmul(f_jacob, P), np.transpose(f_jacob)) + Q
    return P_k1_prior


def update_step(x_hat, x_hat_k1, P_k1, Rn, rho1, rho2, z_k):
    # 5x5
    ep = 10**-8
    
    H = np.array([[ep, 0, 0, 0, 0],
                  [0, ep, 0, 0, 0],
                  [0, 0, ep, rho1, 0],
                  [0, 0, 0, rho2, 0],
                  [0, 0, 0, 0, ep]])
    
    
    # Si = H * P_k1 * H^T + Rn
    Si = np.matmul(np.matmul(H, P_k1), np.transpose(H)) + Rn

    # K_new = P_k1 * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_k1, np.transpose(H)), np.linalg.inv(Si))
    y_new = np.matmul(H, x_hat)

    # 5x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_k1_post = P_k1 - np.matmul(np.matmul(K_new, Si), np.transpose(K_new))


    return x_hat_k1_post, P_k1_post


def get_data_sets(state, fips=None):
    # the post hoc seiir model predictions provided by Prof Flaxman without
    # any kalman filtering:
    seiir = pd.read_csv(r'data/seiir_compartments_post-hoc_ny_state.csv', header=0,
                        index_col='date', parse_dates=True)
    

    fb_data = data_sets.create_symptom_df()
    fb_data_val = data_sets.create_symptom_df(valid=True)

    if fips is None:
        case_data = data_sets.create_case_df_state()
        case_data_geo = case_data.loc[state]['case_rate'].copy()
        fb_data_geo = fb_data.loc[state].groupby('date').sum().copy()
        fb_data_val_geo = fb_data_val.loc[state].groupby('date').sum().copy()

    else:
        case_data = data_sets.create_case_df_county()
        case_data_geo = case_data.loc[fips]['case_rate'].copy()
        
        if fips == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val.loc[(slice(None), '36081'), :].copy()
            fb_data_val_geo = fb_data_val.mean(level='date')
            for borough in nyc_fips:
                fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')
                fb_data_val_geo += fb_data_val.loc[(slice(None), borough), :].copy().mean(level='date')
        else:
            fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()
            fb_data_val_geo = fb_data_val.loc[(slice(None), fips), :].copy()
            # collapse down to a single index column (date)
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val_geo.mean(level='date')


    return seiir, fb_data_geo, fb_data_val_geo, case_data_geo


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()

In [5]:
# set constants
K0 = datetime.date(2020, 4, 12)

the_state = 'NY'
the_county = data_sets.get_fips(the_state, 'Erie')
#the_county = 'New York City'
constants = {
    'alpha': 0.948786,
    'gamma1': 0.500000,
    'gamma2': 0.662215,
    'sigma': 0.266635,
    'theta': 6.000000
    }

# set initial values for Kalman filter parameters
P_mult = 1
Q_mult = 1

# Rn is the R noise matrix; it remains constant thru the stepping of the
# Kalman filter
Rn_mult = 5*10**-8

Rn_22 = 10000
Rn_32 = 1000

Rn_23 = 1000
Rn_33 = 100

Rn = Rn_mult * np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, Rn_22, Rn_23, 0],
                         [0, 0, Rn_32, Rn_33, 0],
                         [0, 0, 0, 0, 0]])

Q = Q_mult * np.eye(5)
P = P_mult * np.eye(5)

# generate data
seiir, fb_data, fb_data_val, case_data = get_data_sets(
    data_sets.STATES[the_state], fips=the_county)


if the_county == 'New York City':
    county_pop = 0
    for each in ['36081', '36005', '36061', '36047', '36085']:
        this_count, state_pop = data_sets.get_pops(each)
        county_pop += this_count
else:
    county_pop, state_pop = data_sets.get_pops(the_county)

b = county_pop / state_pop

# calculate moving averages on the fb and case data
case_ma7 = case_data.rolling(window=7).mean()
case_ma7_all = case_ma7.iloc[6:]
prop_ma7, n_ma7, num_stl_ma7 = calc_fb_ma7(fb_data)

prop_ma7_valid, n_ma7_valid, num_stl_ma7_valid = calc_fb_ma7(fb_data_val)

# get starting compartment values for the state level
x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)

# convert to the county level
x_hat_k0 = b * x_hat_state_k0

I2_county = x_hat_k0[3, 0]


rho1 = 0.0001

rho2 = case_ma7_all.loc['2020-04-12'] / I2_county


# approximate rho values
# I2_county = b * I2
# prop_county = rho1 * I2_county
# case_county = rho2 * I2_county


# create empty dictionaries to hold the estimated values
prop_est = {}
case_est = {}
seiir_pred = {}



# Original data run ----------------
start = K0
d = start
while d <= datetime.date(2020, 9, 30):
#while d <= datetime.date(2020, 8, 25):    

# each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)
    
    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    x_hat_k1 = b * x_hat_state_k1


    # get measurements for current day
    
    z_k = np.array([[0],
                    [0],
                    [prop_ma7.loc[d]],
                    [case_ma7_all.loc[d]],
                    [0]])

    # predict step using the stepped fwd SEIIR compartment values 
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)

    # store estimated values for proportion and case rate
    indexDate = d + datetime.timedelta(days=7)
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    #seiir_pred[indexDate] = b * x_hat_k1[3, 0]
    seiir_pred[indexDate] = x_hat_k1[3, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)

step -----------
P:
[[ 1.13635039 -0.13635039  0.13054686  0.04235332 -0.17290017]
 [-0.13635039  1.20744462 -0.20164108 -0.04235332  0.17290017]
 [ 0.13054686 -0.20164108  1.32109422 -0.25        0.        ]
 [ 0.04235332 -0.04235332 -0.25        1.68852871 -0.43852871]
 [-0.17290017  0.17290017  0.         -0.43852871  1.43852871]]
P_post:
[[-8.88178420e-16  3.05311332e-16  7.77156117e-16 -5.66907632e-15
   2.49800181e-15]
 [ 3.05311332e-16  0.00000000e+00 -8.32667268e-17  6.86950496e-16
  -2.77555756e-16]
 [ 7.49400542e-16 -8.32667268e-17  1.23635367e+00  9.40293877e-08
   9.69574669e-17]
 [-5.64825964e-15  6.73072709e-16  9.40293877e-08  7.10542736e-15
   1.22124533e-15]
 [ 2.49800181e-15 -2.77555756e-16  9.60968151e-17  1.27675648e-15
  -2.22044605e-16]]
step -----------
P:
[[ 1.08320297e+00 -8.32029669e-02  1.60365437e-01 -1.60365421e-01
  -1.61532780e-08]
 [-8.32029669e-02  1.08320297e+00 -1.60365437e-01  1.60365421e-01
   1.61532779e-08]
 [ 1.60365437e-01 -1.60365437e-01  1.309

step -----------
P:
[[ 1.07696659e+00 -7.69665910e-02  1.52417111e-01 -1.52417095e-01
  -1.53526597e-08]
 [-7.69665910e-02  1.07696659e+00 -1.52417111e-01  1.52417095e-01
   1.53526600e-08]
 [ 1.52417111e-01 -1.52417111e-01  1.30183194e+00 -3.01831912e-01
  -3.04029064e-08]
 [-1.52417095e-01  1.52417095e-01 -3.01831912e-01  1.30183188e+00
   3.04029031e-08]
 [-1.53526597e-08  1.53526600e-08 -3.04029064e-08  3.04029031e-08
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00 -1.52655666e-16  1.38777878e-16 -4.71844785e-16
  -1.98523347e-23]
 [-1.24900090e-16 -4.44089210e-16  5.55111512e-17 -1.11022302e-16
  -3.30872245e-24]
 [ 1.11022302e-16  2.77555756e-17  1.20733558e+00  9.18224523e-08
   0.00000000e+00]
 [-4.99600361e-16 -8.32667268e-17  9.18224523e-08  6.43929354e-15
  -2.64697796e-23]
 [-1.98523347e-23 -3.30872245e-24  8.24513782e-18 -2.64697796e-23
   0.00000000e+00]]
step -----------
P:
[[ 1.07678963e+00 -7.67896346e-02  1.52242289e-01 -1.52242273e-01
  -1.53350501e-08]
 [-7.67896346e-

step -----------
P:
[[ 1.07743547e+00 -7.74354717e-02  1.52869618e-01 -1.52869603e-01
  -1.53982399e-08]
 [-7.74354717e-02  1.07743547e+00 -1.52869618e-01  1.52869603e-01
   1.53982400e-08]
 [ 1.52869618e-01 -1.52869618e-01  1.30178831e+00 -3.01788280e-01
  -3.03985113e-08]
 [-1.52869603e-01  1.52869603e-01 -3.01788280e-01  1.30178825e+00
   3.03985082e-08]
 [-1.53982398e-08  1.53982400e-08 -3.03985113e-08  3.03985081e-08
   1.00000000e+00]]
P_post:
[[ 6.66133815e-16 -1.11022302e-16  0.00000000e+00  8.32667268e-17
  -1.65436123e-23]
 [-1.11022302e-16  0.00000000e+00  2.77555756e-17  2.77555756e-17
   3.30872245e-24]
 [ 0.00000000e+00  2.77555756e-17  1.20717835e+00  9.18104953e-08
  -6.61744490e-24]
 [ 5.55111512e-17  0.00000000e+00  9.18104952e-08  6.43929354e-15
   0.00000000e+00]
 [-1.98523347e-23  0.00000000e+00  8.23153236e-18  0.00000000e+00
   0.00000000e+00]]
step -----------
P:
[[ 1.07741090e+00 -7.74109038e-02  1.52846956e-01 -1.52846940e-01
  -1.53959570e-08]
 [-7.74109038e-

step -----------
P:
[[ 1.07987444e+00 -7.98744396e-02  1.55226826e-01 -1.55226810e-01
  -1.56356765e-08]
 [-7.98744396e-02  1.07987444e+00 -1.55226826e-01  1.55226810e-01
   1.56356766e-08]
 [ 1.55226826e-01 -1.55226826e-01  1.30166556e+00 -3.01665528e-01
  -3.03861467e-08]
 [-1.55226810e-01  1.55226810e-01 -3.01665528e-01  1.30166550e+00
   3.03861434e-08]
 [-1.56356765e-08  1.56356766e-08 -3.03861467e-08  3.03861434e-08
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00  0.00000000e+00  2.77555756e-16 -1.38777878e-15
   6.61744490e-24]
 [-2.77555756e-17  2.22044605e-16  2.77555756e-17 -8.32667268e-17
   6.61744490e-24]
 [ 3.05311332e-16  0.00000000e+00  1.20642024e+00  9.17528374e-08
   0.00000000e+00]
 [-1.38777878e-15 -5.55111512e-17  9.17528373e-08  6.88338275e-15
   1.98523347e-23]
 [ 6.61744490e-24  6.61744490e-24  1.25794584e-17  1.32348898e-23
   0.00000000e+00]]
step -----------
P:
[[ 1.08030721e+00 -8.03072079e-02  1.55631167e-01 -1.55631152e-01
  -1.56764050e-08]
 [-8.03072079e-

step -----------
P:
[[ 1.08143287e+00 -8.14328711e-02  1.56675134e-01 -1.56675118e-01
  -1.57815616e-08]
 [-8.14328711e-02  1.08143287e+00 -1.56675134e-01  1.56675118e-01
   1.57815619e-08]
 [ 1.56675134e-01 -1.56675134e-01  1.30143967e+00 -3.01439638e-01
  -3.03633935e-08]
 [-1.56675118e-01  1.56675118e-01 -3.01439638e-01  1.30143961e+00
   3.03633902e-08]
 [-1.57815616e-08  1.57815619e-08 -3.03633934e-08  3.03633901e-08
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00 -1.52655666e-16 -5.55111512e-17  3.33066907e-16
  -1.98523347e-23]
 [-1.38777878e-16  0.00000000e+00 -2.77555756e-17 -2.77555756e-17
  -3.30872245e-24]
 [-8.32667268e-17 -2.77555756e-17  1.20585844e+00  9.17101106e-08
   6.61744490e-24]
 [ 3.88578059e-16 -2.77555756e-17  9.17101106e-08  6.43929354e-15
  -1.32348898e-23]
 [-1.65436123e-23 -3.30872245e-24  2.07259433e-17 -1.32348898e-23
   0.00000000e+00]]
step -----------
P:
[[ 1.08124240e+00 -8.12423964e-02  1.56498266e-01 -1.56498250e-01
  -1.57637460e-08]
 [-8.12423964e-

...

P_post:
[[ 0.00000000e+00  1.94289029e-16 -3.33066907e-16  1.44328993e-15
   4.63221143e-23]
 [ 1.94289029e-16 -2.22044605e-16  8.32667268e-17 -8.32667268e-17
  -6.61744490e-24]
 [-3.05311332e-16  5.55111512e-17  1.19860517e+00  9.11584719e-08
   6.61744490e-24]
 [ 1.44328993e-15 -5.55111512e-17  9.11584719e-08  6.88338275e-15
  -6.61744490e-24]
 [ 4.63221143e-23 -3.30872245e-24 -7.85293510e-18 -6.61744490e-24
   0.00000000e+00]]
step -----------
P:
[[ 1.10572119e+00 -1.05721186e-01  1.77987331e-01 -1.77987313e-01
  -1.79282950e-08]
 [-1.05721186e-01  1.10572119e+00 -1.77987331e-01  1.77987313e-01
   1.79282950e-08]
 [ 1.77987331e-01 -1.77987331e-01  1.29965129e+00 -2.99651262e-01
  -3.01832537e-08]
 [-1.77987313e-01  1.77987313e-01 -2.99651262e-01  1.29965123e+00
   3.01832507e-08]
 [-1.79282950e-08  1.79282950e-08 -3.01832537e-08  3.01832507e-08
   1.00000000e+00]]
P_post:
[[-6.66133815e-16  6.93889390e-17  0.00000000e+00 -1.94289029e-16
   3.30872245e-23]
 [ 6.93889390e-17 -4.440892

step -----------
P:
[[ 1.11749530e+00 -1.17495296e-01  1.87350411e-01 -1.87350392e-01
  -1.88714186e-08]
 [-1.17495296e-01  1.11749530e+00 -1.87350411e-01  1.87350392e-01
   1.88714186e-08]
 [ 1.87350411e-01 -1.87350411e-01  1.29873687e+00 -2.98736835e-01
  -3.00911455e-08]
 [-1.87350392e-01  1.87350392e-01 -2.98736835e-01  1.29873681e+00
   3.00911426e-08]
 [-1.88714186e-08  1.88714186e-08 -3.00911455e-08  3.00911426e-08
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00  3.46944695e-16 -5.55111512e-17  2.22044605e-16
   3.30872245e-24]
 [ 3.60822483e-16 -2.22044605e-16  2.77555756e-16 -1.22124533e-15
  -6.28657266e-23]
 [-5.55111512e-17  2.77555756e-16  1.19477830e+00  9.08674239e-08
   6.61744490e-24]
 [ 2.49800181e-16 -1.22124533e-15  9.08674240e-08  6.43929354e-15
  -1.32348898e-23]
 [ 3.30872245e-24 -5.62482817e-23  4.30188843e-18 -6.61744490e-24
   0.00000000e+00]]
step -----------
P:
[[ 1.11742001e+00 -1.17420013e-01  1.87277123e-01 -1.87277104e-01
  -1.88640364e-08]
 [-1.17420013e-

[[ 1.11732474e+00 -1.17324735e-01  1.87197606e-01 -1.87197587e-01
  -1.88560268e-08]
 [-1.17324735e-01  1.11732474e+00 -1.87197606e-01  1.87197587e-01
   1.88560268e-08]
 [ 1.87197606e-01 -1.87197606e-01  1.29868334e+00 -2.98683309e-01
  -3.00857538e-08]
 [-1.87197587e-01  1.87197587e-01 -2.98683309e-01  1.29868328e+00
   3.00857509e-08]
 [-1.88560268e-08  1.88560268e-08 -3.00857538e-08  3.00857509e-08
   1.00000000e+00]]
P_post:
[[ 4.44089210e-16 -1.38777878e-17  1.11022302e-16 -3.60822483e-16
  -1.98523347e-23]
 [-2.77555756e-17  4.44089210e-16 -5.55111512e-17 -2.77555756e-17
   0.00000000e+00]
 [ 1.11022302e-16 -2.77555756e-17  1.19479352e+00  9.08685814e-08
   0.00000000e+00]
 [-3.60822483e-16  0.00000000e+00  9.08685814e-08  6.88338275e-15
  -1.32348898e-23]
 [-1.98523347e-23  3.30872245e-24  2.58080351e-22 -6.61744490e-24
   0.00000000e+00]]
step -----------
P:
[[ 1.11654366e+00 -1.16543664e-01  1.86578144e-01 -1.86578125e-01
  -1.87936298e-08]
 [-1.16543664e-01  1.11654366e+00 -

step -----------
P:
[[ 1.11846417e+00 -1.18464167e-01  1.88095000e-01 -1.88094981e-01
  -1.89464196e-08]
 [-1.18464167e-01  1.11846417e+00 -1.88095000e-01  1.88094981e-01
   1.89464196e-08]
 [ 1.88095000e-01 -1.88095000e-01  1.29865343e+00 -2.98653395e-01
  -3.00827407e-08]
 [-1.88094981e-01  1.88094981e-01 -2.98653395e-01  1.29865337e+00
   3.00827377e-08]
 [-1.89464196e-08  1.89464196e-08 -3.00827407e-08  3.00827377e-08
   1.00000000e+00]]
P_post:
[[ 4.44089210e-16 -2.77555756e-17 -5.55111512e-17  6.93889390e-16
   3.30872245e-24]
 [ 0.00000000e+00  4.44089210e-16 -8.32667268e-17 -8.32667268e-17
   0.00000000e+00]
 [-2.77555756e-17 -8.32667268e-17  1.19448875e+00  9.08454030e-08
   0.00000000e+00]
 [ 6.66133815e-16 -8.32667268e-17  9.08454029e-08  6.88338275e-15
  -6.61744490e-24]
 [ 9.92616735e-24  0.00000000e+00  4.30010834e-18 -6.61744490e-24
   0.00000000e+00]]
step -----------
P:
[[ 1.11865458e+00 -1.18654583e-01  1.88236265e-01 -1.88236246e-01
  -1.89606489e-08]
 [-1.18654583e-

step -----------
P:
[[ 1.12810985e+00 -1.28109849e-01  1.95441298e-01 -1.95441279e-01
  -1.96863969e-08]
 [-1.28109849e-01  1.12810985e+00 -1.95441298e-01  1.95441279e-01
   1.96863969e-08]
 [ 1.95441298e-01 -1.95441298e-01  1.29816053e+00 -2.98160504e-01
  -3.00330927e-08]
 [-1.95441279e-01  1.95441279e-01 -2.98160504e-01  1.29816047e+00
   3.00330898e-08]
 [-1.96863969e-08  1.96863970e-08 -3.00330928e-08  3.00330898e-08
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00 -1.11022302e-16 -1.11022302e-16  4.99600361e-16
   4.63221143e-23]
 [-8.32667268e-17  0.00000000e+00  5.55111512e-17 -1.11022302e-16
  -6.61744490e-24]
 [-8.32667268e-17  2.77555756e-17  1.19181953e+00  9.06423986e-08
   6.61744490e-24]
 [ 5.55111512e-16 -8.32667268e-17  9.06423985e-08  7.10542736e-15
   1.32348898e-23]
 [ 4.30133919e-23 -6.61744490e-24 -2.36499672e-17  6.61744490e-24
   0.00000000e+00]]
step -----------
P:
[[ 1.12716017e+00 -1.27160168e-01  1.94648383e-01 -1.94648363e-01
  -1.96065283e-08]
 [-1.27160168e-

In [ ]:
# compute squared error for the overlapping time period -------------------------

left = predicted_seiir_prior.index[0]
right = d

seiir_err = case_ma7_all.loc[left:right] - predicted_seiir_prior[left:right]
seiir_sum_sq_err = (seiir_err**2).sum()
seiir_mse = seiir_sum_sq_err / seiir_err.count()

kalman_err = predicted_case.loc[left:right] - case_ma7_all.loc[left:right]
kalman_sum_sq_err = (kalman_err**2).sum()
kalman_mse = kalman_sum_sq_err / kalman_err.count()


print('SSE between seiir forecast and case rate:', seiir_sum_sq_err)
print('MSE between seiir forecast and measured case rate:', seiir_mse)
print()
print('SSE between kalman forecast and case rate:', kalman_sum_sq_err)
print('MSE between kalman forecast and measured case rate:', kalman_mse)

In [ ]:
# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 22})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]
special_start = datetime.date(2020, 8, 16)

week_interval = pd.date_range(start=special_start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
tick_start = K0
tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=tick_start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

In [ ]:
# Single plot with all lines

#matplotlib.rcParams.update({'font.size': 20})
#plt.style.use('seaborn-whitegrid')
#tick_start = K0
#tick_end = predicted_seiir_prior.index[-1]



fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[special_start:d].index, case_ma7_all.loc[special_start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.loc[special_start:tick_end].index, predicted_seiir_prior.loc[special_start:tick_end],
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.loc[special_start:tick_end].index, predicted_case.loc[special_start:tick_end], label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.ylim(-38, 1000)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.xlim(special_start, datetime.date(2020, 10, 9))
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[special_start:d].index, num_stl_ma7.loc[special_start:d], c='red', label='Facebook Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
#plt.plot(prop_ma7.index, num_stl_ma7, c='red', label='Facebook Positive Symptoms',
#         linewidth=width)
#plt.ylim(-.065, 2.5)
plt.ylim(-.4, 11)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')

plt.show()

In [ ]:
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -700
leftylim_high = 20000

rightylim_low = -3
rightylim_high = 85

xtick_size = 14
xlabel_size = 14

In [ ]:
b

In [8]:
# plot findings -- multiple plots

# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 18})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]


fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)




fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax31)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)



fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


(737517.0, 737711.0)

In [ ]:
plt.show()

In [ ]:
b

In [ ]:
# MASE calculation -----------------

left = predicted_case.index[0]
right = predicted_case.index[-1]

# take error between prediction and actual
e = (case_ma7_all.loc[left:right] - predicted_case.loc[left:right]).abs()

case_ma7_diff = case_ma7_all.diff(7)
denom = (case_ma7_diff.loc[left:right]).abs()

term = e.div(denom)
term.replace([np.inf, -np.inf], np.nan, inplace=True)
term.dropna(inplace=True)
mase = term.sum() / len(term)
print('mase:', mase)

In [ ]:
# SHORTENED Erie plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -72
leftylim_high = 900

rightylim_low = -1
rightylim_high = 12.5

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED Albany plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -20
leftylim_high = 275

rightylim_low = -.225
rightylim_high = 3.1

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED Westchester plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -100
leftylim_high = 900

rightylim_low = -.75
rightylim_high = 7

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED Nassau plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -110
leftylim_high = 1200

rightylim_low = -1.1
rightylim_high = 12

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED NYC plot settings

xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -295
leftylim_high = 8000

rightylim_low = -3
rightylim_high = 80

xtick_size = 14
xlabel_size = 14

In [ ]:
# NYC plot settings

xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -700
leftylim_high = 12000

rightylim_low = -5
rightylim_high = 85

xtick_size = 14
xlabel_size = 14

In [ ]:
# Nassau plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -40
leftylim_high = 1650

rightylim_low = -.3
rightylim_high = 12

xtick_size = 14
xlabel_size = 14

In [ ]:
# Westchester plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -28
leftylim_high = 1000

rightylim_low = -.2
rightylim_high = 7

xtick_size = 14
xlabel_size = 14

In [ ]:
# Albany plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -3
leftylim_high = 75

rightylim_low = -.125
rightylim_high = 3.1

xtick_size = 14
xlabel_size = 14

In [6]:
# Erie plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -10
leftylim_high = 250

rightylim_low = -.5
rightylim_high = 12.5

xtick_size = 14
xlabel_size = 14